In [1]:
import datetime as dt

import numpy as np
import pandas as pd
from freqtrade.exchange import timeframe_to_minutes
from keras import Sequential
from keras.losses import categorical_crossentropy
from lazyft.data_loader import load_pair_data

from constants import IMAGES_PATH, REPO
import keras
from keras.initializers import initializers_v2 as initializer

from preprocess import preprocess
from constants import IMAGES_PATH
from cnn_model import create_cnn
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizer_v2.adam import Adam
import tensorflow as tf

tf.config.run_functions_eagerly(True)

In [ ]:
pair = "BTC/USDT"
SPLIT = 0.30
LR = 0.001
TIMESTAMP = dt.datetime.now().strftime("%Y%m%d%H%M%S")
# cnn_networks = 1
models: list[keras.Sequential] = []
image_train_path = TRAIN_IMAGES_PATH / pair.replace("/", "_")
image_test_path = TEST_IMAGES_PATH / pair.replace("/", "_")
image_train_path.mkdir(parents=True, exist_ok=True)
image_test_path.mkdir(parents=True, exist_ok=True)
preprocess(
    timerange="20170101-20211231",
    image_save_path=image_train_path,
    pair=pair,
    interval="1h",
)
preprocess(
    timerange="20220101-", pair=pair, image_save_path=image_test_path
)

# global history, string_list, summary
target_size = 40
batch_size = 20
EPOCHS = 10

In [7]:


initializers_ = [
    initializer.Orthogonal(),
    initializer.LecunUniform(),
    initializer.VarianceScaling(),
    initializer.RandomNormal(),
    initializer.RandomUniform(),
    initializer.TruncatedNormal(),
    initializer.GlorotNormal(),
    initializer.GlorotUniform(),
    initializer.HeNormal(),
    initializer.HeUniform(),
    initializer.Orthogonal(seed=42),
    initializer.LecunUniform(seed=42),
    initializer.VarianceScaling(seed=42),
    initializer.RandomNormal(seed=42),
    initializer.RandomUniform(seed=42),
    initializer.TruncatedNormal(seed=42),
    initializer.GlorotNormal(seed=42),
    initializer.GlorotUniform(seed=42),
    initializer.HeNormal(seed=42),
    initializer.HeUniform(seed=42),
]
# for i, initializer_ in enumerate(initializers_):
#     cnn = create_cnn(target_size, kernel_initializer=initializer_)
#     # Compile each model
#     cnn.compile(optimizer=Adam(learning_rate=LR), loss='binary_crossentropy', metrics=['acc'])
#     models.append(cnn)

# All images will be rescaled by 1./255
train_validate_datagen = ImageDataGenerator(
    rescale=1 / 255, validation_split=SPLIT
)  # set validation split
test_datagen = ImageDataGenerator(rescale=1 / 255)
# data_chunks = ensemble_data(len(models), str(image_path))
test_generator = test_datagen.flow_from_directory(
    directory=image_test_path,
    target_size=(target_size, target_size),
    batch_size=batch_size,
)

2022-04-03 09:45:23.250 | INFO     | lazyft.downloader:download_missing_historical_data:301 - Checking if download is needed for BTC/USDT @ 1h interval(s)
2022-04-03 09:45:23.253 | INFO     | lazyft.downloader:download_missing_historical_data:338 - Data is up to date
2022-04-03 09:45:23.315 | INFO     | lazyft.data_loader:load_pair_data:39 - Loaded 35041 rows for BTC/USDT, data starts at 2018-01-01 00:00:00+00:00
Generating images...
========PREPROCESS REPORT========:
Total Data Points: 1461
Total Images Created: 1441
Total LONG positions: 700
Total SHORT positions: 741
2022-04-03 09:45:32.889 | INFO     | lazyft.downloader:download_missing_historical_data:301 - Checking if download is needed for BTC/USDT @ 1h interval(s)
2022-04-03 09:45:32.891 | INFO     | lazyft.downloader:download_missing_historical_data:338 - Data is up to date
2022-04-03 09:45:32.940 | INFO     | lazyft.data_loader:load_pair_data:39 - Loaded 2209 rows for BTC/USDT, data starts at 2022-01-01 00:00:00+00:00
Generat

In [3]:
def load_model(pair: str, time: str, model_name: str) -> Sequential:
    model = create_cnn(40)
    model_to_load = REPO / pair.replace("/", "_") / time / "models" / model_name
    model.load_weights(model_to_load)

    return model


model = load_model(pair, "20220403092229", "HeUniform.h5")
model.compile(
    optimizer=Adam(learning_rate=LR), loss=categorical_crossentropy, metrics=["acc"]
)

In [11]:
scores = model.evaluate(test_generator)
print("Test {0}s: {1:.2f}%".format(model.metrics_names[1], scores[1] * 100))
string_list = []
model.summary(print_fn=lambda x: string_list.append(x))
string_list.append(f"test acc: {scores[1] * 100}")


1/4 [======>.......................] - ETA: 0s - loss: 0.6396 - acc: 0.7500

/home/raphael/PycharmProjects/freqgym/venv/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 [==============================] - 0s 109ms/step - loss: 0.6582 - acc: 0.6712
Test accs: 67.12%


In [13]:
scores

[0.6582338213920593, 0.6712328791618347]

In [15]:
sample = test_generator.next()

In [16]:
test_generator.class_indices


{'LONG': 0, 'SHORT': 1}

In [2]:

btc = load_pair_data("BTC/USDT", timeframe="1h", timerange="20220401-")

2022-04-08 16:13:02.069 | INFO     | lazyft.downloader:download_missing_historical_data:302 - Checking if download is needed for BTC/USDT @ 1h interval(s)
2022-04-08 16:13:02.072 | INFO     | lazyft.downloader:download_missing_historical_data:339 - Data is up to date
2022-04-08 16:13:02.136 | INFO     | lazyft.data_loader:load_pair_data:40 - Loaded 186 rows for BTC/USDT, data starts at 2022-04-01 00:00:00+00:00


In [ ]:
btc.groupby()

In [3]:
from util import group_by
from datetime import timedelta
import time


w = btc.head(480)

# t1 = time.perf_counter()
# sampled = btc.resample('4H').mean()
# print('Elapsed time:', timedelta(seconds=time.perf_counter() - t1))


In [5]:
w.groupby(pd.Grouper(key='date', freq='d')).get_group()

{Timestamp('2022-04-01 00:00:00+0000', tz='UTC', freq='D'): 24,
 Timestamp('2022-04-02 00:00:00+0000', tz='UTC', freq='D'): 48,
 Timestamp('2022-04-03 00:00:00+0000', tz='UTC', freq='D'): 72,
 Timestamp('2022-04-04 00:00:00+0000', tz='UTC', freq='D'): 96,
 Timestamp('2022-04-05 00:00:00+0000', tz='UTC', freq='D'): 120,
 Timestamp('2022-04-06 00:00:00+0000', tz='UTC', freq='D'): 144,
 Timestamp('2022-04-07 00:00:00+0000', tz='UTC', freq='D'): 168,
 Timestamp('2022-04-08 00:00:00+0000', tz='UTC', freq='D'): 186}

In [20]:
w.set_index("date").resample("1d").mean()

,open,high,low,close,volume
date,,,,,
2022-01-01 00:00:00+00:00,47.1K,47.3K,46.9K,47.1K,816.853
2022-01-02 00:00:00+00:00,47.2K,47.4K,47.1K,47.2K,764.186
2022-01-03 00:00:00+00:00,46.8K,47K,46.6K,46.8K,1.15K
2022-01-04 00:00:00+00:00,46.4K,46.6K,46.2K,46.4K,1.48K
2022-01-05 00:00:00+00:00,46K,46.2K,45.6K,45.9K,2.16K
2022-01-06 00:00:00+00:00,43.1K,43.3K,42.9K,43.1K,1.62K
2022-01-07 00:00:00+00:00,42.1K,42.3K,41.7K,42K,2.28K
2022-01-08 00:00:00+00:00,41.7K,41.9K,41.5K,41.7K,1.37K
2022-01-09 00:00:00+00:00,41.9K,42.1K,41.7K,41.9K,946.850


In [2]:
from freqtrade.exchange import timeframe_to_minutes

MINUTES_IN_DAY = 1440
tf_minutes = timeframe_to_minutes("5m")
20 * MINUTES_IN_DAY // tf_minutes


5760

In [26]:
btc.head(1).index[0]

0

In [3]:
import util
from preprocess import alive_bar


def split_timeframes(
    df: pd.DataFrame,
    window: int,
    timeframes: list[str],
    list_dates: list[str],
    download_interval: str,
) -> dict[str, list[pd.Series]]:
    """
    Given a dataframe, a window size, a list of timeframes, and a list of dates,
    split the dataframe into a list of lists of series, where each series is a list of prices
    for a specific timeframe

    :param df: The dataframe that contains the data
    :param window: The size of the moving window. This is the number of observations used for
    calculating the statistic. Each window will be a fixed size
    :param timeframes: a list of timeframes to use for grouping the data
    :param list_dates: list of dates
    :return: A list of lists of series. Each list of series is a list of the closing prices of the
    stocks for a given timeframe.
    """
    series_dict: dict[str, list[pd.Series]] = {}
    index = window
    with alive_bar(
        len(list_dates) - window - 1, title="Creating decision map...", bar="filling"
    ) as bar:
        while True:
            idx = index
            save_idx = index - 1

            if idx == len(list_dates):
                break
            if index >= len(list_dates) - 1:
                save_idx = -2
                after = df["date"] > list_dates[-window - 1]
                data_slice = df.loc[after]
            else:
                after = df["date"] > list_dates[idx - window]
                before = df["date"] <= list_dates[idx]
                data_slice = df.loc[after & before]

            # select appropriate timeframe
            # print('len of data slice: ', len(data_slice), 'head: ', data_slice.head())
            tf_series = []

            # group dataslices by timeframe
            for freq in timeframes:
                if freq == download_interval:
                    tf_series.append(data_slice["close"].tail(20))
                    continue
                group_dt = util.group_by(data_slice, freq)
                tf_series.append(group_dt["close"].tail(20))
            series_dict[list_dates[save_idx]] = tf_series
            index += 1
            bar()
    return series_dict



In [19]:
btc = load_pair_data("BTC/USDT", timeframe="1h", timerange="20220313-")

2022-04-07 06:52:45.380 | INFO     | lazyft.downloader:download_missing_historical_data:301 - Checking if download is needed for BTC/USDT @ 1h interval(s)
2022-04-07 06:52:45.382 | INFO     | lazyft.downloader:download_missing_historical_data:338 - Data is up to date
2022-04-07 06:52:45.438 | INFO     | lazyft.data_loader:load_pair_data:40 - Loaded 581 rows for BTC/USDT, data starts at 2022-03-13 00:00:00+00:00


In [ ]:
split_timeframes(
    btc, 20, ["1h", "4h", "8h", "1d"], util.get_dates_from_df(btc, "1d"), "1h"
)

In [20]:
series_dict: dict[str, list[pd.Series]] = {}
window = 2
df = btc
list_dates = util.get_dates_from_df(btc, "1d")
timeframes = ["1h", "4h", "8h", "1d"]
download_interval = "1h"
index = window

In [15]:
idx = index
save_idx = index - 1

assert idx != len(list_dates)

if index >= len(list_dates) - 1:
    save_idx = -2
    after = df["date"] > list_dates[-window - 1]
    data_slice = df.loc[after]
else:
    after = df["date"] > list_dates[idx - window]
    before = df["date"] <= list_dates[idx]
    data_slice = df.loc[after & before]

# select appropriate timeframe
# print('len of data slice: ', len(data_slice), 'head: ', data_slice.head())
tf_series = []

# group dataslices by timeframe
for freq in timeframes:
    if freq == download_interval:
        tf_series.append(data_slice["close"].tail(20))
        continue
    group_dt = util.group_by(data_slice, freq)
    tf_series.append(group_dt["close"].tail(20))
series_dict[list_dates[save_idx]] = tf_series
index += 1

In [24]:
df.loc[(df["date"] > list_dates[20 - 20]) & (df["date"] < list_dates[20])]

,date,open,high,low,close,volume
1,2022-03-13 01:00:00+00:00,38.9K,39.3K,38.9K,39K,1.29K
2,2022-03-13 02:00:00+00:00,39K,39.2K,39K,39.2K,649.294
3,2022-03-13 03:00:00+00:00,39.2K,39.2K,39.1K,39.2K,604.576
4,2022-03-13 04:00:00+00:00,39.2K,39.3K,39.1K,39.1K,547.678
5,2022-03-13 05:00:00+00:00,39.1K,39.2K,39.1K,39.1K,318.244
...,...,...,...,...,...,...
475,2022-04-01 19:00:00+00:00,46.3K,46.4K,46.1K,46.4K,1.36K
476,2022-04-01 20:00:00+00:00,46.4K,46.4K,46.1K,46.1K,889.165
477,2022-04-01 21:00:00+00:00,46.1K,46.4K,46.1K,46.3K,704.819
478,2022-04-01 22:00:00+00:00,46.3K,46.4K,46.2K,46.4K,1.04K
